In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
warnings.filterwarnings('ignore')
data = pd.read_csv('train.csv')
train = data.sample(frac=0.7, random_state=1)
test = data[data.index.isin(train.index)==False]
#train['Obs']=range(1,805)  # maybe change index from 1 to 804
train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SalePrice
8,20,RL,NaN,6853,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2009,WD,220000
791,160,FV,30.0,3180,Pave,Pave,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,151000
571,20,RL,60.0,6960,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,Shed,500,11,2009,WD,120500
224,20,RL,70.0,8050,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,3,2007,WD,127000
498,20,RL,NaN,9556,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,8,2007,WD,160000


In [184]:
rows, columns = train.shape
print(rows)
print(columns)

563
79


In [185]:
train.describe().round(2) # without object type

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemod/Add,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,563.00,451.00,563.00,563.00,563.00,563.00,563.00,560.00,563.00,563.00,...,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00,563.00
mean,56.86,68.29,9800.64,5.97,5.63,1970.00,1982.61,103.04,439.10,54.17,...,90.55,41.44,25.37,1.73,14.31,1.02,21.76,6.06,2007.91,173867.17
std,41.65,22.60,5832.44,1.32,1.08,27.55,20.78,189.73,410.48,186.25,...,121.59,59.17,64.31,17.14,49.66,24.28,223.55,2.58,1.30,69561.87
min,20.00,21.00,1300.00,2.00,3.00,1880.00,1950.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,2006.00,35000.00
25%,20.00,56.00,7490.00,5.00,5.00,1953.00,1963.50,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.00,2007.00,129000.00
50%,50.00,69.00,9259.00,6.00,5.00,1970.00,1990.00,0.00,404.00,0.00,...,0.00,16.00,0.00,0.00,0.00,0.00,0.00,6.00,2008.00,156000.00
75%,70.00,80.00,11366.50,7.00,6.00,1997.00,2002.00,148.00,695.50,0.00,...,166.00,62.00,0.00,0.00,0.00,0.00,0.00,7.00,2009.00,206500.00
max,190.00,155.00,70761.00,10.00,9.00,2008.00,2009.00,1224.00,1965.00,1526.00,...,646.00,365.00,364.00,219.00,287.00,576.00,4500.00,12.00,2010.00,538000.00


In [186]:
# variable preparation

predictors=list(train.columns[2:-1])
continuous_predictors=train.select_dtypes(['int64']).columns
columns_categorical=train.select_dtypes(['object']).columns


## Step 1: Categorial variables clean and discussion

In [187]:
train['MSZoning'].unique()

array(['RL', 'FV', 'RM', 'RH', 'C (all)', 'I (all)'], dtype=object)

In [188]:
# these variables are part of 'columns_categorical', 
categorical=['MSZoning', 'Street','LotShape', 'LandContour','Utilities','LotConfig', 'LandSlope'
           , 'Neighborhood', 'Condition1', 'Condition2' ,'BldgType','HouseStyle', 'RoofStyle', 'RoofMatl', 
             'Exterior1st','Exterior2nd','ExterQual', 'ExterCond', 'Foundation','Heating'
            , 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual','Functional'
            , 'PavedDrive','SaleType']


train['MSZoning']=train['MSZoning'].str.replace('(',"")
train['MSZoning']=train['MSZoning'].str.replace(')',"")
for item1 in categorical:
    train[item1]=train[item1].str.replace(' ',"")
    length=len(train[item1].unique())
    count=0
    while (length-1)>=count:
        for item2 in train[item1].unique():
            count1=str(count)
            train[item1]=train[item1].str.replace(item2,count1)
            count=int(count1)+1

In [189]:
#This is to replace NaN with empty string for now
train = train.replace(np.nan, '', regex=True)

#Use this to replace whatever first argument is with second argument
#train['Alley'] = train['Alley'].replace(0,'' , regex=True)


### Only 3 values with pool variables, we can just drop this

In [190]:
del train['PoolQC']
del train['PoolArea']
del test['PoolQC']
del test['PoolArea']

In [191]:
df = pd.DataFrame({'Fence':list(train['Fence'])})
pd.get_dummies(df).head()
#Here, the base case Fence means the house does not have a 

,Fence_,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


23 places have a feature out of 805 observations. Is it worthwhile to even include? Only 2% of observations have this

Therefore, I decided to remove it.

In [192]:
del train['MiscFeature']
del test['MiscFeature']

In [193]:
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,Fence,MiscVal,MoSold,YrSold,SaleType,SalePrice
8,20,0,,6853,0,,0,0,0,0,...,25,0,0,0,,0,6,2009,0,220000
791,160,1,30,3180,0,Pave,1,0,0,0,...,166,0,0,0,,0,6,2006,0,151000
571,20,0,60,6960,0,,1,0,0,0,...,0,0,0,0,,500,11,2009,0,120500
224,20,0,70,8050,0,,1,0,0,0,...,64,0,0,0,,0,3,2007,0,127000
498,20,0,,9556,0,,0,0,0,1,...,21,0,0,0,,0,8,2007,0,160000


In [194]:
del train['Alley']
del test['Alley']

In [195]:
#Need to conver any string into integer for machine learning algorithm to work
combined_set = pd.concat([train, test], axis = 0)

In [196]:
for feature in combined_set.columns: # Loop through all columns in the dataframe
    if combined_set[feature].dtype == 'object': # Only apply for columns with categorical strings
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes # Replace strings with an integer

In [198]:
final_train = combined_set[:train.shape[0]] # Up to the last initial training set row
final_test = combined_set[train.shape[0]:] # Past the last initial training set row

In [199]:
#This is the price variable we need to remove
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')

# Part of code where I'm actually doing things.

In [190]:
data = pd.read_csv('train1.csv')
final_train = data.sample(frac=0.7, random_state=1)
final_test = data[data.index.isin(final_train.index)==False]
final_train.head()
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')

In [182]:
import numpy
import pandas
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Time for some Adaptive boosting with Decision Trees

In [157]:
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit


### Testing to see if I can actually use Adaboost

In [49]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

#http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor
#http://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_regression.html

regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=9),random_state=0)
regr.fit(final_train,y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=9, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=50, random_state=0)

In [50]:
from sklearn.model_selection import cross_val_score
#This uses cross validation to evaluate the score
cross_val_score(regr, final_train, y_train, cv=10)

array([ 0.99932327,  0.99959537,  0.99332714,  0.9892081 ,  0.98621175,
        0.99369846,  0.96178037,  0.99895646,  0.9972338 ,  0.99883252])

# Bringing it all together to find optimal level of nodes to use!

In [ ]:
values = range(1,20)
maximum = 0
optimal_node = 0
for i in values:
    regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=i),random_state=0)
    print("Cross val score for {} nodes".format(i))
    print(cross_val_score(regr, final_train, y_train, cv=10))
    tot = 0
    g = cross_val_score(regr, final_train, y_train, cv=10)
    #Check to see if this layer is better since highest cross_val_score
    for x in g:
        tot = tot+x
    if tot>maximum:
        optimal_node = i
       

In [51]:
from sklearn.metrics import r2_score
print("Optimal nodes is {}".format(19))
regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=19),random_state=0)
print("Cross val score for {} nodes".format(19))
print(cross_val_score(regr, final_train, y_train, cv=10))

regr.fit(final_train, y_train)
y_1 = regr.predict(final_test)
accuracy = r2_score(y_test, y_1)
print("Cross-Predicted Accuracy: {}".format(accuracy))

Optimal nodes is 19
Cross val score for 19 nodes
[ 0.9992743   0.99972439  0.9918568   0.9988606   0.98562704  0.99372742
  0.96162895  0.99874786  0.99619191  0.99933667]
Cross-Predicted Accuracy: 0.9841112164939263


In [52]:
from QBUS2820 import rmse_jack, r2_jack 
#Jackknife method to be able to report the standard errors for the test results

columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = ['Results']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, y_1)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, y_1))
results.round(3)

,Test RMSE,SE,Test R2,SE
Results,8876.037,6800.324,0.984,0.015


# Here we create an esemble by using Lasso in addition to it

In [53]:
from sklearn.linear_model import LassoCV

lasso = LassoCV(cv=10)
lasso.fit(final_train, np.ravel(y_train)) 
pred_L = lasso.predict(final_test)

In [54]:
#The decision tree with AdaBoosting
regr.fit(final_train, y_train)
pred_T = regr.predict(final_test)

In [55]:
#Combining the 2 predictions
pred_combined = (pred_L+pred_T)/2

In [56]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = ['Lasso', 'Adaboost Tree', 'Lasso + Tree']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, pred_L)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, pred_L))
results.iloc[1,0], results.iloc[1,1] = rmse_jack(y_test, pred_T)
results.iloc[1,2], results.iloc[1,3] = (r2_jack(y_test, pred_T))
results.iloc[2,0], results.iloc[2,1] = rmse_jack(y_test, pred_combined)
results.iloc[2,2], results.iloc[2,3] = (r2_jack(y_test, pred_combined))

results.round(3)

,Test RMSE,SE,Test R2,SE
Lasso,31352.429,2953.435,0.802,0.027
Adaboost Tree,8876.037,6800.324,0.984,0.015
Lasso + Tree,17789.751,3203.686,0.936,0.017


# Worked! Let's try combining ridge and Enet too

In [57]:
#Elastic Net
from sklearn.linear_model import ElasticNetCV
enet = ElasticNetCV(l1_ratio=[0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99], cv=5)
enet.fit(final_train, np.ravel(y_train))

from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=enet.alpha_, l1_ratio=enet.l1_ratio_)
enet.fit(final_train, np.ravel(y_train))

ElasticNet(alpha=177395.46992995607, copy_X=True, fit_intercept=True,
      l1_ratio=0.98999999999999999, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [58]:
#Ridge Regression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

alphas = np.exp(np.linspace(-10,20,500)) 
ridge = RidgeCV(alphas=alphas, cv=5)
ridge.fit(final_train, np.ravel(y_train))
ridge = Ridge(alpha=ridge.alpha_)
ridge.fit(final_train, np.ravel(y_train))

Ridge(alpha=9.3405800180084659e-05, copy_X=True, fit_intercept=True,
   max_iter=None, normalize=False, random_state=None, solver='auto',
   tol=0.001)

In [59]:
pred_R = ridge.predict(final_test)
pred_L = lasso.predict(final_test)
pred_E = enet.predict(final_test)

In [60]:
5

In [83]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE', 'MSE']
rows = ['Lasso', 'Adaboost Tree', 'Lasso + Tree', 'Ridge', 'Enet', 'Tree + Ridge', 'Tree + Enet', 'Tree + All 3']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, pred_L)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, pred_L))
results.iloc[1,0], results.iloc[1,1] = rmse_jack(y_test, pred_T)
results.iloc[1,2], results.iloc[1,3] = (r2_jack(y_test, pred_T))
results.iloc[2,0], results.iloc[2,1] = rmse_jack(y_test, pred_combined)
results.iloc[2,2], results.iloc[2,3] = (r2_jack(y_test, pred_combined))
results.iloc[3,0], results.iloc[3,1] = rmse_jack(y_test, pred_R)
results.iloc[3,2], results.iloc[3,3] = (r2_jack(y_test, pred_R))
results.iloc[4,0], results.iloc[4,1] = rmse_jack(y_test, pred_E)
results.iloc[4,2], results.iloc[4,3] = (r2_jack(y_test, pred_E))
results.iloc[5,0], results.iloc[5,1] = rmse_jack(y_test, pred_combined2)
results.iloc[5,2], results.iloc[5,3] = (r2_jack(y_test, pred_combined2))
results.iloc[6,0], results.iloc[6,1] = rmse_jack(y_test, pred_combined3)
results.iloc[6,2], results.iloc[6,3] = (r2_jack(y_test, pred_combined3))
results.iloc[7,0], results.iloc[7,1] = rmse_jack(y_test, pred_combined4)
results.iloc[7,2], results.iloc[7,3] = (r2_jack(y_test, pred_combined4))
results.iloc[0,4] = mean_absolute_error(y_test, pred_L)
results.iloc[1,4] = mean_absolute_error(y_test, pred_T)
results.iloc[2,4] = mean_absolute_error(y_test, pred_combined)
results.iloc[3,4] = mean_absolute_error(y_test, pred_R)
results.iloc[4,4] = mean_absolute_error(y_test, pred_E)
results.iloc[5,4] = mean_absolute_error(y_test, pred_combined2)
results.iloc[6,4] = mean_absolute_error(y_test, pred_combined3)
results.iloc[7,4] = mean_absolute_error(y_test, pred_combined4)
results.round(3)

,Test RMSE,SE,Test R2,SE,MSE
Lasso,31352.429,2953.435,0.802,0.027,21162.760
Adaboost Tree,8876.037,6800.324,0.984,0.015,1355.697
Lasso + Tree,22338.679,2745.458,0.899,0.017,14808.837
Ridge,6634.128,1103.772,0.991,0.002,3908.118
Enet,32884.354,2760.191,0.782,0.027,22877.886
Tree + Ridge,6864.211,3718.343,0.990,0.007,2335.568
Tree + Enet,18459.672,3043.717,0.931,0.017,11639.225
Tree + All 3,17680.982,2502.215,0.937,0.013,11272.559


# From this, it appears that Ridge + Tree is the best Model

# Let's try Tree + PCR

In [62]:
from QBUS2820 import pcrCV
pcr=pcrCV(final_train, y_train)

In [63]:
pred_pcr = pcr.predict(final_test)

In [64]:
#The decision tree with AdaBoosting
regr.fit(final_train, y_train)
pred_T = regr.predict(final_test)

In [65]:
pred_combined = (pred_pcr+pred_T)/2

In [66]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = ['PCR', 'Adaboost Tree', 'PCR + Tree']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, pred_pcr)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, pred_pcr))
results.iloc[1,0], results.iloc[1,1] = rmse_jack(y_test, pred_T)
results.iloc[1,2], results.iloc[1,3] = (r2_jack(y_test, pred_T))
results.iloc[2,0], results.iloc[2,1] = rmse_jack(y_test, pred_combined)
results.iloc[2,2], results.iloc[2,3] = (r2_jack(y_test, pred_combined))

results.round(3)

,Test RMSE,SE,Test R2,SE
PCR,32713.469,2469.051,0.784,0.025
Adaboost Tree,8876.037,6800.324,0.984,0.015
PCR + Tree,18313.077,2861.478,0.932,0.015


In [67]:
#Adding lasso back now
pred_L = lasso.predict(final_test)

In [68]:
pred_combined = (pred_pcr+pred_T+pred_L)/3

In [69]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = [ 'LASSO + PCR + Tree']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, pred_combined)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, pred_combined))

results.round(3)

,Test RMSE,SE,Test R2,SE
LASSO + PCR + Tree,22338.679,2745.458,0.899,0.017


# Lasso + Tree seems to do better without dimension reduction

# Trying out model stacking now where we combine both lasso and tree to be predictors in an OLS regression

In [70]:
#The decision tree with AdaBoosting
regr.fit(final_train, y_train)
pred_T = regr.predict(final_test)

In [71]:
#Adding lasso back now
pred_L = lasso.predict(final_test)

In [72]:
print(len(final_train))
print(len(y_train))
print(len(final_test))
print(len(y_test))

402
402
402
402


In [73]:
import numpy as np
import pandas as pd

stacked_lasso_tree = pd.DataFrame({'Lasso':pred_L, 'Tree':pred_T, 'Price':y_test})
#Created new dataframe with both predictions as values

In [74]:
import statsmodels.formula.api as sm
result = sm.ols(formula="Price ~ Lasso + Tree", data=stacked_lasso_tree).fit()

In [75]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                 1.282e+04
Date:                Fri, 01 Sep 2017   Prob (F-statistic):               0.00
Time:                        11:31:26   Log-Likelihood:                -4217.7
No. Observations:                 402   AIC:                             8441.
Df Residuals:                     399   BIC:                             8453.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3869.6459   1341.656     -2.884      0.004   -6507.243   -1232.049
Lasso         -0.0036      0.017     -0.217      0.828      -0.036       0.029
Tree           1.0272      0.015     68.883      0.000       0.998       1.056
==============================================================================
Omnibus:                      901.017   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1813411.956
Skew:                          17.213   Prob(JB):                         0.00
Kurtosis:                     330.228   Cond. No.                     8.20e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.2e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [76]:
predictions = result.predict(exog=stacked_lasso_tree) 

In [82]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE', 'MSE']
rows = [ 'Stacked Model with Tree and LASSO']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, predictions)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, predictions))
results.iloc[0,4] = mean_absolute_error(y_test, predictions)
results.round(3)

,Test RMSE,SE,Test R2,SE,MSE
Stacked Model with Tree and LASSO,8715.002,6036.584,0.985,0.014,1914.723


In [78]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, predictions)

1914.7233520277218

# Prediction on data

In [202]:
testdata = pd.read_csv('Test1.csv')

In [203]:
del testdata['BoxCoxPrice']
np.any(np.isnan(testdata))
np.all(np.isfinite(testdata))
testdata = testdata.replace(np.nan, 0, regex=True)

In [252]:
from sklearn.metrics import r2_score
print("Optimal nodes is {}".format(20))
regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=19),random_state=0)

regr.fit(final_train, y_train)
y_1 = regr.predict(y_test)

Optimal nodes is 20


ValueError: Expected 2D array, got 1D array instead:
array=[ 210000.  192000.  287000.  145500.  177625.  163000.  215200.  119900.
  207500.  275000.  200000.  224500.  275000.  145000.  129000.  148000.
  194500.  250000.  185000.   78000.  200000.  319900.  205000.  122500.
   90350.  153000.  180000.  148000.  125000.  221800.  125500.  143250.
  259000.  157500.   90000.  189000.  286000.  139000.  410000.  148000.
  188000.  246900.  140000.  157000.  180500.  137500.  162900.  370000.
  214900.   96000.  215000.  173000.  100000.  204000.  132000.  183000.
  172000.  170000.  260000.  191000.  195000.  138000.  262500.  462000.
  315500.  190000.  157000.  136000.  160500.  110000.  191000.  100000.
  148000.  188000.  134450.  211500.  285000.   95000.  137000.  174000.
  134500.  190000.  127000.  250000.  170000.   99500.  160000.  122500.
  140000.  203000.  169000.   82000.  135000.  175500.  240000.  153575.
  184500.  105000.  137500.  140000.  230000.  164900.  137500.  

In [233]:
len(final_test)

241

In [234]:
#mean_absolute_error(y_test, y_1)
#(r2_jack(y_test, y_1))

In [235]:
len(y_1)

1608

In [236]:
#This is for the indices
ind = np.arange(1,1609)

In [237]:
headers = ['Id','Prediction']
predictions = pd.DataFrame({'Id':ind, 'Prediction':y_1})
predictions

,Id,Prediction
0,1,297000.0
1,2,297000.0
2,3,297000.0
3,4,297000.0
4,5,297000.0
5,6,297000.0
6,7,297000.0
7,8,297000.0
8,9,297000.0
9,10,297000.0


In [144]:
#Saving results into CSV file 
predictions.to_csv("Predictions.csv", index=False)

In [222]:
#Ridge Regression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

alphas = np.exp(np.linspace(-10,20,500)) 
ridge = RidgeCV(alphas=alphas, cv=5)
ridge.fit(final_train, np.ravel(y_train))
ridge = Ridge(alpha=ridge.alpha_)
ridge.fit(final_train, np.ravel(y_train))

predictionR = ridge.predict(testdata)

In [223]:
predict = (predictionR+y_1)/2

In [224]:
headers = ['Id','PredictionR']
predictions = pd.DataFrame({'Id':ind, 'Prediction':predict})
predictions

,Id,Prediction
0,1,7.428515e+09
1,2,1.625666e+10
2,3,6.638699e+09
3,4,1.073268e+10
4,5,1.372699e+10
5,6,2.369111e+10
6,7,1.338415e+10
7,8,1.681577e+10
8,9,6.189492e+09
9,10,3.711441e+09


In [152]:
#Saving results into CSV file 
predictions.to_csv("Predictions2.csv", index=False)

In [225]:
#Elastic Net
from sklearn.linear_model import ElasticNetCV
enet = ElasticNetCV(l1_ratio=[0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99], cv=5)
enet.fit(final_train, np.ravel(y_train))

from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=enet.alpha_, l1_ratio=enet.l1_ratio_)
enet.fit(final_train, np.ravel(y_train))

ElasticNet(alpha=166961.3766010862, copy_X=True, fit_intercept=True,
      l1_ratio=0.98999999999999999, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [227]:
stuff = enet.predict(testdata)

In [228]:
stuff

array([  87636.32732361,   98446.03178341,  102835.16405021, ...,
         52542.35023086,  100976.85632371,  115962.45590171])

In [229]:
headers = ['Id','PredictionR']
predictions = pd.DataFrame({'Id':ind, 'Prediction':stuff})
predictions

,Id,Prediction
0,1,87636.327324
1,2,98446.031783
2,3,102835.164050
3,4,119776.634304
4,5,99383.663769
5,6,180215.112390
6,7,73859.715653
7,8,148969.695539
8,9,69273.854773
9,10,75592.983166


In [230]:
from sklearn.linear_model import LassoCV

lasso = LassoCV(cv=10)
lasso.fit(final_train, np.ravel(y_train)) 
pred_L = lasso.predict(testdata)

In [231]:
headers = ['Id','Prediction']
predictions = pd.DataFrame({'Id':ind, 'Prediction':pred_L})
predictions

,Id,Prediction
0,1,88163.514822
1,2,93974.254135
2,3,94970.531242
3,4,122574.276843
4,5,104263.027099
5,6,171173.272047
6,7,72558.631532
7,8,153961.078682
8,9,66994.723503
9,10,77307.061878


In [248]:
pp = (pred_L +stuff)/2

In [249]:
for x in np.nditer(pp, op_flags=['readwrite']):
    if x<0:
          x[...] = np.average(pp)

In [250]:
headers = ['Id','Prediction']
predictions = pd.DataFrame({'Id':ind, 'Prediction':pp})
predictions

,Id,Prediction
0,1,87899.921073
1,2,96210.142959
2,3,98902.847646
3,4,121175.455574
4,5,101823.345434
5,6,175694.192219
6,7,73209.173592
7,8,151465.387110
8,9,68134.289138
9,10,76450.022522


In [251]:
#Saving results into CSV file 
predictions.to_csv("Predictions2.csv", index=False)